# 2: The Image Classification Dataset

In [1]:
%matplotlib inline
import time
import torch
import torchvision
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display()

## Dataset

In [ ]:
class FashionMNIST(d2l.DataModule):     #@save
    """The Fashion-MNIST dataset."""
    def __init__(self, batch_size = 64, resize = (28, 28)):
        super().__init__()
        self.save_hyperparameters()
        trans = transforms.Compose([transforms.Resize(resize),
                                    transforms.ToTensor()])
        self.train = torchvision.datasets.FashionMNIST(
            root = self.root, train=True, transform=trans, download=True
        )